In [1]:
import os
import time
import json

from utils.file_utils import *
from datasets import list_datasets
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from IPython import get_ipython
import pandas as pd
from tqdm import tqdm
# from Sai_Test.MergeModel import *
# from Sai_Test.ClassifierModel import *
import matplotlib.pyplot as plt
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification
from datasets import load_dataset
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from MergeModel import *
from ClassifierModel import *

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', 2)

In [2]:
!ls

BART_classifier_final.pt      __pycache__
classifier_model.pt	      save_train_imgs.mat
ClassifierModel.py	      save_train_labels.mat
classifier_model_senticlf.pt  Sentiment_Classifier.ipynb
experiment		      Test.ipynb
Grid_search.ipynb	      Test_results_bart_model.ipynb
MergeModel.py		      Test_results_merged_model.ipynb
MergeModel_Train.ipynb	      utils


In [6]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
vocab_size = tokenizer.vocab_size

In [8]:
sent_model = ClassifierModel(vocab_size, 64, 2, 2, 512).cuda()
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').cuda()

In [9]:
merge_model = MergeModel(summary_model, sent_model)

In [10]:
merge_model.load_state_dict(torch.load('experiment/BART_classifier_final.pt'))
merge_model = merge_model.cuda()

CLF

In [7]:
dataset = load_dataset("imdb")

Reusing dataset imdb (/tmp/xdg-cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [8]:
train_batch = dataset['train']['text']
train_labels = torch.tensor(dataset['train']['label'])
val_batch = dataset['test']['text']
val_labels = torch.tensor(dataset['test']['label'])

In [9]:
seq_length = 150
input_encoding = tokenizer(train_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)
val_encoding = tokenizer(val_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)

In [10]:
input_ids = input_encoding['input_ids']
input_mask = input_encoding['attention_mask']

In [11]:
val_ids = val_encoding['input_ids']
val_mask = val_encoding['attention_mask']

SUMMARY

In [11]:
df = pd.read_csv("../BART/test.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 11,490



,article,highlights,id
8922,The second in command at the sheriff's departm...,Tim Albin of the Tulsa County Sheriff's Office...,c02c4dd532284e36ac6fb158ad58b359acbb9f42
10628,Tottenham target Kevin Wimmer will move to Eng...,Cologne defender Kevin Wimmer is a £4.4million...,eaa38cd30802e3df64d8b8251292de205cf88c0b
5720,A Briton jailed in Burma for using an image of...,Phil Blackwood was sentenced to two-and-a-half...,6fe7f20e98833deea2cc54f26571496101788a49
9334,Activated charcoal is thought to act as a 'det...,Dietitian Sophie Claessens picks the latest 'm...,ca3ed0691af054a8b3ab95593b8ec81e5ebfdeab
8529,Just days after Bruce Jenner's powerful interv...,Scott Turner Schofield has landed a role on th...,b69be56fc67aa3734fd3fff5f4673893d5bc0434
1025,"Ali Addeh refugee camp, Djibouti (CNN)Henol an...","For 25 years, Ali Addeh refugee camp has been ...",f588be66f67386056c4f771af390a1845a702415
695,"(CNN)Mark Ronson's ""Uptown Funk!,"" featuring B...",The song rules the chart for 13th week .\nIt p...,a43893ac704663476a285673392de84c0c17d4e5
2241,This is the moment a heroic teenage Baskin Rob...,"Rachael Bishop, 19, at first pleaded with the ...",1bb70c912148cb8b7aa7247bcb723602d864ef68
4778,Thierry Henry has responded to critics who dis...,Javier Hernandez scored a late goal to secure ...,5907dc116da34857e9dc0b647769afb3f0338c95
8523,England wing Christian Wade has been slammed b...,The Matt Hampson foundation held an event on W...,b6865e6147f9e5c5fa962d0f67b8f4bddf0e8899


In [12]:
train_sentence = df.article.values
train_target = df.highlights.values


num_data_points = 500
train_sentence = list(train_sentence)[:num_data_points]
train_target = list(train_target)[:num_data_points]

In [13]:
article_encoding = tokenizer(train_sentence, return_tensors='pt', padding=True, truncation = True, max_length=500)
summary_encoding = tokenizer(train_target, return_tensors='pt', padding=True,truncation = True, max_length=100)

In [14]:
article_input_ids = article_encoding['input_ids']
article_attention_mask = article_encoding['attention_mask']

In [15]:
summary_input_ids = summary_encoding['input_ids']
summary_attention_mask = summary_encoding['attention_mask']

In [16]:
batch_size = 1

train_data = TensorDataset(input_ids, input_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader_clf = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

NameError: name 'input_ids' is not defined

In [17]:
batch_size = 4
train_data = TensorDataset(article_input_ids, article_attention_mask,\
                           summary_input_ids, summary_attention_mask)

In [18]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [19]:
from nltk.translate.bleu_score import sentence_bleu

In [20]:
def get_bleu(reference, candidate):
    score1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))    
    return score1

In [21]:
c1,c2,c3 = next(iter(train_dataloader_clf)) 
c1 = c1.cuda() 
c2 = c2.cuda() 
c3 = c3.cuda() 

NameError: name 'train_dataloader_clf' is not defined

In [60]:
b1,b2,b3,b4 = next(iter(train_dataloader)) 
b1 = b1.cuda() 
b2 = b2.cuda() 
b3 = b3.cuda() 
b4 = b4.cuda()

In [61]:
tokenizer.decode(b1[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

'(CNN)The leader of Yemen\'s Houthi rebels vowed not to back down on Sunday as a top Saudi military official claimed weeks of airstrikes had significantly weakened the Shiite group. "Our fighters will not evacuate from the main cities or the government institutions," rebel leader Abdul-Malik al-Houthi said in a televised address. "Anyone who thinks we will surrender is dreaming." His comments came after more than three weeks of Saudi-led coalition bombings aimed at pushing back the Houthis, who surged into the capital of Sanaa in January and ousted President Abdu Rabu Mansour Hadi from power. Hadi still claims he\'s Yemen\'s legitimate leader and is working with the Saudis and other allies to return to his country. Since mid-March, more than 700 people have been killed in violence that shows no sign of slowing, according to figures from the World Health Organization. The Houthis say they have the support of the country\'s people behind them. "It\'s the right of the Yemeni people to sta

In [62]:
tokenizer.decode(b3[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

"Abdul-Malik al-Houthi says in a televised address that fighters will not pull out of major cities.\nA top military leader pledges allegiance to Yemen's ousted President."

In [63]:
summary_out = merge_model.summary_model.generate(b1, num_beams=1, max_length = 100, early_stopping=True)
summary_out

tensor([[    2,     0, 13112,   417,   922,    12, 18764,   967,  1076,    12,
           725, 15617,   118,   161,    11,    10, 14451,  1100,    14,  5464,
            40,    45,  2999,    66,     9,   538,  1947,   479, 50118,   250,
           299,   831,   884, 19629, 24614,     7,  6519,    18, 14716,   270,
           479,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1],
        [    2,     0, 28012,  9396,  3217,    35,   188,  5680, 17004, 16597,
          7052,    40,   860,     7,   120, 12620,     7,  1477,   435,  3115,
           479, 50118,   894,   161,  4766,   403,   924,   350,   171,   400,
          6522,  2025,    75,   562,  1579,   479, 50118, 34052,   611,    40,
            33,     7,   342,   159,     5, 28488,     6,  1339,    62,     5,
          4757,     8,   146,   699,    14,     5,  1659,   641,  9731, 

In [64]:
tokenizer.decode(summary_out[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

"Abdul-Malik al-Houthi says in a televised address that fighters will not pull out of major cities.\nA top military leader pledges allegiance to Yemen's ousted President."

In [65]:
merge_model.load_state_dict(torch.load('experiment/BART_classifier_final.pt'))
merge_model = merge_model.cuda()

In [29]:
sent_model.load_state_dict(torch.load('experiment/classifier_model_senticlf.pt'))
sent_model = sent_model.cuda()
sent_model = sent_model.eval()

In [33]:
temp = []
ss = []

for b1, b2, b3, b4 in train_dataloader:
    merge_model.eval()
    b1 = b1.cuda() 
    b2 = b2.cuda() 
    b3 = b3.cuda() 
    b4 = b4.cuda()
    summary_out,*sentiments = merge_model(b1, b2, b3, b4)
    ss.append(torch.abs(sent_model(summary_out.logits.argmax(dim = -1)) - sent_model(b1)/6).mean().item())
    
    for i in range(b3.shape[0]):
        summary_gt = tokenizer.decode(b3[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summary_pred = tokenizer.decode(summary_out.logits.argmax(dim = -1)[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        temp.append(get_bleu(summary_pred, summary_gt))
        
print(np.array(temp).mean(), np.array(ss).mean())

0.175555125627341 0.03472723811864853


In [34]:
merge_model.load_state_dict(torch.load('experiment/BART_classifier_final.pt'))
merge_model = merge_model.cuda()

In [35]:
temp = []
ss = []

for b1, b2, b3, b4 in train_dataloader:
    merge_model.eval()
    b1 = b1.cuda() 
    b2 = b2.cuda() 
    b3 = b3.cuda() 
    b4 = b4.cuda()
    summary_out,*sentiments = merge_model(b1, b2, b3, b4)
    ss.append(torch.abs(sent_model(summary_out.logits.argmax(dim = -1)) - sent_model(b1)/6).mean().item())
    
    for i in range(b3.shape[0]):
        summary_gt = tokenizer.decode(b3[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summary_pred = tokenizer.decode(summary_out.logits.argmax(dim = -1)[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        temp.append(get_bleu(summary_pred, summary_gt))
        
print(np.array(temp).mean(), np.array(ss).mean())

0.175555125627341 0.03472723811864853
